# From Word Embeddings To Document Distances详解
> 《From Word Embeddings To Document Distances》 review

- toc: true 
- badges: true
- comments: true
- author: Bujie Xu
- categories: [NLP]

## 1. 简介

《From Word Embeddings To Document Distances》这篇文章，发表于2014年。作者在word2vec的基础上提出了一种衡量文章相似度的尺度，Word Mover's Distance(WDM)。 WDM用来描述两篇文章的词向量之间的距离。这里的“距离”作者用了“旅行距离”来描述，意思是从一篇文章的词向量转换到另一篇文章的词向量的最短距离。在此距离的基础上，作者提出了衡量文章相似度的算法。从实验可以看出，该方法优于当时的SOTA。

下文会按照作者的思路来详细介绍WDM的定义和文章相似度的计算方法（站在作者角度来想）：

- 在计算文章相似度的问题中，前人的方法只考虑词频，没有考虑文章的意义，把word2vec用到文章相似度的评价里会不会有所提高呢？
- 那么我来定义了一种新的计算文章相似度的方法WMD。
- 可是WMD的计算复杂度太高。那我提出计算下限的方法WCD和RWMD。
- 有了WCD和RWMD，然后通过Prefetch和Prune来找到一个文章的k nearest neighbors。
- 看看我的实验结果多牛叉。超过了当时的SOTA。

## 2. 论文详解

### 一、Word2Vec Embedding

在2013年，Mikolov et al提出了word2vec，这个在当时引起了很大反响。该研究用向量来表示每个单词，从数学上更好的表示了单词的意义。传统上表示文章，大多数都是用BOW或者TFIDF的手法，这些手法对于词的意义有欠考虑，所以在衡量文章相似度上，往往停留在单词一致性的表层上，文章意思的相似性得不到很大的表现。作者关注到word2vec的发展，然后将该方法应用到了文章相似度的表达上。这里我们首先介绍以下word2vec。

简单来说，word2vec是一种通过学习神经网络来寻找词向量表示的一种方法。具体来说word2vec的skip-gram模型是通过构建一个单层神经网络（input layer, projection layer, output layer)来预测每个单词的相邻单词。通过学习该神经网络，得到的projection layer即为每个单词的词向量。训练目标是使每个单词的相邻单词的出现概率最大化。该概率可以用下式表示
$$
\frac{1}{T}\sum_{t=1}^T\sum_{j \in nb(t)}logp(w_j|w_t) \tag{1}
$$

这里T为单词数，nb(t)表示单词$w_t$的相邻单词，$p(w_j|w_t)$用hierarchical softmax来提高训练速度。 



### 二、Word Move's Distance

首先我们用词向量来定义Word travel cost。假设$x_i, x_j$表示word $i$和word $j$的词向量。我们用$c(i, j) = ||x_i - x_j||_2$来表示从一个词到另一个词的"旅行距离"（Word travel cost）。有了词于词之间的距离，接下来我们来定义文章间距离。设$d$, $d'$为两篇文章的nBow (normalized bag of words)表示。令$T_{ij}, T \in R^{n\times n}$来表示文章d的词i到文章d'的距离。我们令词i到文章d'的所有词的距离之和为$\sum_jT_{ij} = d_i$, 反之文章d'的词j到文章d的所有词之和为$\sum_iT_{ij}=d'_j$。最终，我们定义两个文章的距离为从一篇文章d到另一篇文章d‘距离的加权累加的最小值。用数学描述为
$$
\min_{T>=0}\sum_{i,j=1}^nT_{ij}c(i,j)　\\
subject \ to: \sum_{j=1}^nT_{i,j} = d_i, \forall_i \in \{1,...,n\} \\
\sum_{i=1}^nT_{i,j} = d'_j, \forall_j \in \{1,...,n\} \tag{2}
$$
所以，求解文章与文章之间的WMD转化为了一个最优化问题。

通过下图简要举例介绍词数相同时文章间距离和词数不同时文章距离的情况。在上半部分，去除了（the, to ,in, a）等等stopwords之后，D0,D1,D2都是由4个不同的词构成的。所以，所有的词的$d_i=0.25$，这里的箭头表示的是$T_{ij}c(i,j)$, 由于词向量空间上，比起band，Obama离President更近，这里的分数也很好的反应了该结果。在图的下半部分，我们可以看出，当词数不同时，一个词可能会映射到多个相似的词。

![WMD1](images/WMD1.png)



### 三、类比运输问题 

运输问题的典型情况是研究单一品种物质的运输调度问题：设某种物品有m个产地$A_1，A_2，···，A_m$，各产地的产量分别是$a_1，a_2，···，a_m$,有n个销地$B_1，B_2，···，B_n$，各个销地的销量分别为$b_1，b_2，···，b_n$。假定从产地$A_i(i=1,2,···,m)$向销地$B_j(j=1,2,···,n)$运输单位物品的运价为$c_{ij}$，怎么调运这些物品才能使总运费最小？可以看出，文章的WMD计算和运输问题是完全对应的。这里产地$A_1，A_2，···，A_m$对应于文章A的单词，$a_1，a_2，···，a_m$为单词的词频。同理$B_1，B_2，···，B_n$为文章B的单词，$b_1，b_2，···，b_n$为对应的词频。运价$c_{ij}$对应于文章A的单词和文章B的单词间的欧拉距离。运输问题可以描述成以下线性规划问题。

$$
\min_z = \sum_{i=1}^m\sum_{j=1}^nc_{ij}x_{ij} \\
subject \ to: \sum_{j=1}^nx_{i,j} = a_i, \forall_i \in \{1,...,n\} \\
\sum_{i=1}^nx_{i,j} = b_j, \forall_j \in \{1,...,n\}  \\
x_{ij} \geq 0 \tag{3}
$$

### 四、Fast Distance Computation

解决上述WMD最优化问题的复杂度时$O(p^3\log p)$, p为去重后单词数。可以想象，当文章有很多单词时，计算WMD会变的非常困难。为此，作者提出了两种求解WMD下限的方法

##### WCD：通过三角不等式$||x+y|| \leq ||x|| + ||y||$，我们可以简单的得到

$$
\sum_{i,j=1}^nT_{ij}c(i,j) = \sum_{i,j=1}^nT_{ij}||x_i-x'_j||_2 \\
= \sum_{i,j=1}||T_{ij}(x_i-x'_j)||_2 \geq ||\sum_{i,j=1}^nT_{ij}(x_i-x'_j)||_2 \\
= ||\sum_{i=1}^n(\sum_{j=1}^nT_{ij})x_i-\sum_{j=1}^n(\sum_{j=1}^nT_{ij})x'_j||_2 \\
= ||\sum_{i=1}^nd_ix_i - \sum_{j=1}^nd'_jx'_j||_2 \tag{4}
$$

作者将这个距离称为Word Centroid Distance（WCD），可以看出计算这个WMD的下限WCD非常快，算法复杂度是O(dp), d为词向量的维度，p为去重后的单词数。在寻找某篇文章的k个最相似的文章时，WCD可以用于寻找有利的候补，从而提高算法效率。

##### RWMD：虽然WCD很容易计算，但是因为他给出的下限太低，作者在这里提出了一种更加逼近真实值的下限Relaxed word moving distance。

想法很简单，就是去掉一个WMD的约束条件。假设去除第二个约束条件，原式变成
$$
\min_{T>=0}\sum_{i,j=1}^nT_{ij}c(i,j)　\\
subject \ to: \sum_{j=1}^nT_{i,j} = d_i, \forall_i \in \{1,...,n\} \tag{5}
$$
因为在所有满足WMD的解之中，肯定也可以找到一个满足RWMD的解，所以RWMD可以作为WMD的一个下限。从直观上理解，去掉约束条件后，文章B的词语并不一定都要被映射到，我们只要让每一个文章A的词语都旅行到文章B即可（举个极端的例子，文章A的所有词语都映射到文章B的某一个词）。类比于运输问题，即是我们只对产量有要求，对销量没有要求。在这种情况下，可想而知，我们只要让文章A的所有词都映射到离其词向量最近的点即可得到最优解。
$$
T_{ij}^* = \begin{cases}
d_{i}\ if j=argmin_{j}c(i,j) \\
0 \ otherwise.
\end{cases} \tag{6}
$$
可以得到
$$
\sum_jT_{ij}c(i,j) \geq \sum_jT_{ij}^*c(i,j) \tag{7}
$$
因为计算RWMD只需要找个离每个词语的最近的词，他的复杂度是$O(p^2)$ 

### 五、Prefetch and prune

作者利用上述WCD和RWMD提出了一种快速查找某文章的k nearest neighbors的算法。

1. 计算所有文章对该文章的WCD，然后按升序排序
2. 计算前k个文章的WMD（计算k nearest neighbors的WMD）
3. 然后来计算剩余文章的RWMD，如果某文章的RWMD大于第k个文章的WMD，那么就不再考虑该文章。反之，我们更新k nearest neighbors



### 六、结果
通过以下结果可知，这种计算文章距离的算法击败了当时大部分的SOAT。
![WDM2](images/WMD2.png)

## 3. 思考

在有了word2vec后，计算文章距离的想法其实很容易就能够想到。不过如果没有近似算法，那么就没有实际利用价值。做科研，理论和实践两手都要硬啊。

{{ '[From Word Embeddings To Document Distances](http://proceedings.mlr.press/v37/kusnerb15.pdf)' | fndetail: 1 }}
{{ '[百度百科：运输问题](https://baike.baidu.com/item/%E8%BF%90%E8%BE%93%E9%97%AE%E9%A2%98/12734790?fr=aladdin)' | fndetail: 2 }}